We are going to extract data from redshift and carry out the following transformation tasks
- (Y) remove all rows where customer id is missing - **SQL** or Python
- (Y) remove certain stock codes - **SQL** or Python
- (Y) add description to the online transactions table - **SQL** or Python
- (Y) remove all duplicate data - **Python**
- (Y) replace missing stock description with Unknown  - SQL or **Python**
- (Y) fix data type - **Python** or SQL


In [101]:
# import the libraries we need

import psycopg2
import pandas as pd

import boto3
from io import StringIO

# add if you want to remove warning messages
import warnings
warnings.filterwarnings("ignore")


## Extracting Data

In [102]:
# NEVER share passwords
# this only works with a .env file, you can hardcode the values for now

import os

from dotenv import load_dotenv
load_dotenv()  # only for local testing


dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [103]:
# definition that connects to redshift

def connect_to_redshift(dbname, host, port, user, password):
    """definition to connect to redshift"""

    connect = psycopg2.connect(
            dbname=dbname, host=host, port=port, user=user, password=password
        )
    
    print("connection to redshift made")
    
    return connect

In [104]:
connect = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [105]:
# check everything is working

query = """select *
from bootcamp.stock_description
limit 10
"""

pd.read_sql(query, connect)

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
5,10124G,ARMY CAMO BOOKCOVER TAPE
6,10125,MINI FUNKY DESIGN TAPES
7,10133,COLOURING PENCILS BROWN TUBE
8,10135,COLOURING PENCILS BROWN TUBE
9,11001,ASSTD DESIGN RACING CAR PEN


In [106]:
# how many rows does the online transaction table contain??

query = """select *
           from bootcamp.online_transactions
"""

online_trans = pd.read_sql(query, connect)

In [107]:
online_trans.shape

# this is important, because when we do a left join our new table should have the same number of rows as our "left table"

(541910, 7)

In [108]:
# joins the description field to the online transactions table
# removes rows of data where customer id is blank
# removes rows of data where stock code is in bank charges, postages etc...

query = """select ot.*,
                  sd.Description
           from bootcamp.online_transactions as ot
           left join bootcamp.stock_description as sd on ot.stock_code = sd.stock_code
           where ot.customer_id <> ''
            and ot.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
"""

online_trans_option1 = pd.read_sql(query, connect)

In [109]:
online_trans_option1.shape

# because we have some stock codes with two descriptions we have to add a sub query to fix that
# however, if you find that is too complicated please stick to this stock

(412922, 8)

In [110]:
# joins the description field to the online transactions table
# removes invoices where customer id is blank
# removes invoices where stock code is in bank charges, postages etc...

query = """
select o.*,
       s.description
from bootcamp.online_transactions o
/*this is a sub query that removes question marks from the stock desc table*/
left join (select *
          from bootcamp.stock_description
          where description <> '?') s on o.stock_code = s.stock_code
where o.customer_id <> ''
    and o.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
"""

online_trans_w_desc = pd.read_sql(query, connect)
online_trans_w_desc.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536368,22914,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BLUE COAT RACK PARIS FASHION
2,536367,48187,4,2010-12-01 08:34:00,7.95,u13047,United Kingdom,DOORMAT NEW ENGLAND
3,536370,22726,12,2010-12-01 08:45:00,3.75,u12583,France,ALARM CLOCK BAKELIKE GREEN
4,536375,82482,6,2010-12-01 09:32:00,2.10,u1785,United Kingdom,WOODEN PICTURE FRAME WHITE FINISH


In [111]:
online_trans_w_desc.shape

# this removes all cases where the description in the stock description table has a ?

(405063, 8)

In [112]:
online_trans_w_desc[online_trans_w_desc.customer_id == '']

# check: we have no cases of missing customer ids

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description


In [113]:
online_trans_w_desc[online_trans_w_desc.stock_code == 'POSTAGE']

# check: we have no cases of stock codes with postage

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description


In [114]:
type(online_trans_w_desc)

pandas.core.frame.DataFrame

In [115]:
online_trans_w_desc.shape

# dropped from 541k rows to 405k rows, and increased the number of columns by one

(405063, 8)

## Transforming Data

### Removing duplicated data

In [116]:
# task 1 - Remove duplicated data, from the first table

online_trans_option1.duplicated().sum()

5303

In [117]:
# task 1 - Remove duplicated data

online_trans_w_desc.duplicated().sum()

5222

In [118]:
online_trans_w_desc[online_trans_w_desc.duplicated(keep=False)].sort_values(by=["invoice", "stock_code"]).head(10)

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
889,536409,21866,1,2010-12-01 11:45:00,1.25,u17908,United Kingdom,UNION JACK FLAG LUGGAGE TAG
1997,536409,21866,1,2010-12-01 11:45:00,1.25,u17908,United Kingdom,UNION JACK FLAG LUGGAGE TAG
14720,536409,22111,1,2010-12-01 11:45:00,4.95,u17908,United Kingdom,SCOTTIE DOG HOT WATER BOTTLE
16856,536409,22111,1,2010-12-01 11:45:00,4.95,u17908,United Kingdom,SCOTTIE DOG HOT WATER BOTTLE
1995,536409,22866,1,2010-12-01 11:45:00,2.10,u17908,United Kingdom,HAND WARMER SCOTTY DOG DESIGN
15883,536409,22866,1,2010-12-01 11:45:00,2.10,u17908,United Kingdom,HAND WARMER SCOTTY DOG DESIGN
2484,536409,22900,1,2010-12-01 11:45:00,2.95,u17908,United Kingdom,SET 2 TEA TOWELS I LOVE LONDON
2485,536409,22900,1,2010-12-01 11:45:00,2.95,u17908,United Kingdom,SET 2 TEA TOWELS I LOVE LONDON
37,536412,21448,2,2010-12-01 11:49:00,1.65,u1792,United Kingdom,12 DAISY PEGS IN WOOD BOX
313,536412,21448,1,2010-12-01 11:49:00,1.65,u1792,United Kingdom,12 DAISY PEGS IN WOOD BOX


In [119]:
# code to only keep the first appearance of a duplicated row

online_trans_cleaned = online_trans_w_desc.drop_duplicates(keep = 'first')
online_trans_cleaned.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536368,22914,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BLUE COAT RACK PARIS FASHION
2,536367,48187,4,2010-12-01 08:34:00,7.95,u13047,United Kingdom,DOORMAT NEW ENGLAND
3,536370,22726,12,2010-12-01 08:45:00,3.75,u12583,France,ALARM CLOCK BAKELIKE GREEN
4,536375,82482,6,2010-12-01 09:32:00,2.10,u1785,United Kingdom,WOODEN PICTURE FRAME WHITE FINISH


In [120]:
# no duplicated rows of data anymore

online_trans_cleaned.duplicated().sum()

0

In [121]:
print(online_trans_w_desc.shape)
print(online_trans_cleaned.shape)

(405063, 8)
(399841, 8)


In [122]:
online_trans_w_desc[(online_trans_w_desc.invoice == "536409") &
                   (online_trans_w_desc.stock_code == "21866")]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
889,536409,21866,1,2010-12-01 11:45:00,1.25,u17908,United Kingdom,UNION JACK FLAG LUGGAGE TAG
1997,536409,21866,1,2010-12-01 11:45:00,1.25,u17908,United Kingdom,UNION JACK FLAG LUGGAGE TAG


In [123]:
online_trans_cleaned[(online_trans_cleaned.invoice == "536409") &
                   (online_trans_cleaned.stock_code == "21866")]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
889,536409,21866,1,2010-12-01 11:45:00,1.25,u17908,United Kingdom,UNION JACK FLAG LUGGAGE TAG


In [124]:
# if you did not remove rows where stock description is ?, you can double check the numbers here

online_trans_cleaned_1 = online_trans_option1.drop_duplicates(keep='first')
print(online_trans_option1.shape)
print(online_trans_cleaned_1.shape)

(412922, 8)
(407619, 8)


### Fixing the invoice date data type

In [125]:
online_trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399841 entries, 0 to 405062
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       399841 non-null  object 
 1   stock_code    399841 non-null  object 
 2   quantity      399841 non-null  int64  
 3   invoice_date  399841 non-null  object 
 4   price         399841 non-null  float64
 5   customer_id   399841 non-null  object 
 6   country       399841 non-null  object 
 7   description   398669 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 27.5+ MB


In [126]:
# transformation task # 2 - fix the invoice date data type

online_trans_cleaned.invoice_date = pd.to_datetime(online_trans_cleaned.invoice_date)
online_trans_cleaned.invoice_date

0        2010-12-01 08:26:00
1        2010-12-01 08:34:00
2        2010-12-01 08:34:00
3        2010-12-01 08:45:00
4        2010-12-01 09:32:00
                 ...        
405058   2011-12-09 12:19:00
405059   2011-12-09 12:19:00
405060   2011-12-09 12:20:00
405061   2011-12-09 12:31:00
405062   2011-12-09 12:50:00
Name: invoice_date, Length: 399841, dtype: datetime64[ns]

In [127]:
online_trans_cleaned.info()

# the invoice date is now a data type datetime64

<class 'pandas.core.frame.DataFrame'>
Index: 399841 entries, 0 to 405062
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       399841 non-null  object        
 1   stock_code    399841 non-null  object        
 2   quantity      399841 non-null  int64         
 3   invoice_date  399841 non-null  datetime64[ns]
 4   price         399841 non-null  float64       
 5   customer_id   399841 non-null  object        
 6   country       399841 non-null  object        
 7   description   398669 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.5+ MB


In [128]:
online_trans_cleaned.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536368,22914,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BLUE COAT RACK PARIS FASHION
2,536367,48187,4,2010-12-01 08:34:00,7.95,u13047,United Kingdom,DOORMAT NEW ENGLAND
3,536370,22726,12,2010-12-01 08:45:00,3.75,u12583,France,ALARM CLOCK BAKELIKE GREEN
4,536375,82482,6,2010-12-01 09:32:00,2.10,u1785,United Kingdom,WOODEN PICTURE FRAME WHITE FINISH


### Replacing missing description values with Unknown 

In [129]:
online_trans_cleaned.isna().sum()

invoice            0
stock_code         0
quantity           0
invoice_date       0
price              0
customer_id        0
country            0
description     1172
dtype: int64

In [130]:
online_trans_cleaned[online_trans_cleaned.description.isnull()]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
339,536500,46000M,10,2010-12-01 12:35:00,1.55,u17377,United Kingdom,None
751,536557,22686,1,2010-12-01 14:41:00,1.25,u17841,United Kingdom,None
942,536408,21705,12,2010-12-01 11:41:00,1.65,u14307,United Kingdom,None
1040,536595,21705,5,2010-12-01 17:24:00,1.65,u13576,United Kingdom,None
1136,536597,21703,4,2010-12-01 17:35:00,0.42,u18011,United Kingdom,None
...,...,...,...,...,...,...,...,...
398631,581514,21705,84,2011-12-09 11:20:00,0.39,u17754,United Kingdom,None
398855,581469,21704,1,2011-12-08 19:28:00,0.85,u14606,United Kingdom,None
398884,581516,21705,24,2011-12-09 11:26:00,0.39,u14422,United Kingdom,None
403625,580438,46000U,10,2011-12-04 12:17:00,1.25,u12827,United Kingdom,None


In [131]:
# replace missing descriptions with Unknown

online_trans_cleaned.description.fillna("Unknown", inplace = True)

In [132]:
# check there are no missing descriptions

online_trans_cleaned.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
description     0
dtype: int64

In [133]:
# check the shape 

online_trans_cleaned.shape

(399841, 8)

In [134]:

online_trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399841 entries, 0 to 405062
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       399841 non-null  object        
 1   stock_code    399841 non-null  object        
 2   quantity      399841 non-null  int64         
 3   invoice_date  399841 non-null  datetime64[ns]
 4   price         399841 non-null  float64       
 5   customer_id   399841 non-null  object        
 6   country       399841 non-null  object        
 7   description   399841 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.5+ MB


In [135]:
# check the shape - if you did not remove the stock descriptions with ?

# fixes the data type of the invoice date
online_trans_cleaned_1.invoice_date = pd.to_datetime(online_trans_cleaned_1.invoice_date)

# replaces where description is unknown
online_trans_cleaned_1.description.fillna("Unknown", inplace = True)
print(online_trans_cleaned_1.shape)

(407619, 8)


In [137]:
online_trans_cleaned_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407619 entries, 0 to 412921
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       407619 non-null  object        
 1   stock_code    407619 non-null  object        
 2   quantity      407619 non-null  int64         
 3   invoice_date  407619 non-null  datetime64[ns]
 4   price         407619 non-null  float64       
 5   customer_id   407619 non-null  object        
 6   country       407619 non-null  object        
 7   description   407619 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 28.0+ MB


## Loading Data to s3

In [138]:
AWS_ACCESS_KEY_ID = os.getenv("aws_access_key_id")
AWS_SECRET_ACCESS_KEY = os.getenv("aws_secret_access_key_id")
AWS_S3_BUCKET = 'july-bootcamp'

In [148]:
# i am using definitions to do this step, which i will introduce y'all to. Feel free to use the slides.

import boto3
from io import StringIO, BytesIO

def connect_to_s3(aws_access_key_id, aws_secret_access_key):
    """Methods that connects to s3"""

    s3_client = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )

    print("Connection to s3 made")
    
    return s3_client


def df_to_s3(df, key, s3_bucket, aws_access_key_id, aws_secret_access_key):
    """Function that writes a data frame as a .csv or .pkl file to a s3 bucket"""
    
    file_type = key[-4:]
    
    if file_type == '.pkl':
        buffer = BytesIO()  # create buffer to temporarily store the Data Frame
        df.to_pickle(buffer)  # code to write the data frame as .pkl file
    
    if file_type == '.csv':
        buffer = StringIO()  # create buffer to temporarily store the Data Frame
        df.to_csv(buffer, index=False)  # code to write the data frame as .csv file

    s3_client = connect_to_s3(aws_access_key_id, aws_secret_access_key)

    s3_client.put_object(
        Bucket=s3_bucket, Key=key, Body=buffer.getvalue()
    )  # this code writes the temp stored file and writes to s3


    print(f"The transformed data is saved as {file_type} in the following location s3://{s3_bucket}/{key}")

In [149]:
key = 'friday_transformations/sh_online_transactions_v2.pkl'

df_to_s3(online_trans_cleaned, key, AWS_S3_BUCKET, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

Connection to s3 made
The transformed data is saved as .pkl in the following location s3://july-bootcamp/friday_transformations/sh_online_transactions_v2.pkl


In [150]:
key = 'friday_transformations/sh_online_transactions_v2.csv'

df_to_s3(online_trans_cleaned, key, AWS_S3_BUCKET, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

Connection to s3 made
The transformed data is saved as .csv in the following location s3://july-bootcamp/friday_transformations/sh_online_transactions_v2.csv


In [154]:
# check - reading an s3 file

key = 'friday_transformations/sh_online_transactions_v2.pkl'

s3_client = connect_to_s3(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

response = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=key)

# if you are reading a pickle file use read pickle, otherwise use read csv
check = pd.read_pickle(response.get("Body"))
#check = pd.read_csv(response.get("Body"))

check

Connection to s3 made


,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536368,22914,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom,BLUE COAT RACK PARIS FASHION
2,536367,48187,4,2010-12-01 08:34:00,7.95,u13047,United Kingdom,DOORMAT NEW ENGLAND
3,536370,22726,12,2010-12-01 08:45:00,3.75,u12583,France,ALARM CLOCK BAKELIKE GREEN
4,536375,82482,6,2010-12-01 09:32:00,2.10,u1785,United Kingdom,WOODEN PICTURE FRAME WHITE FINISH
...,...,...,...,...,...,...,...,...
405058,581579,20713,10,2011-12-09 12:19:00,1.79,u17581,United Kingdom,JUMBO BAG OWLS
405059,581579,23293,8,2011-12-09 12:19:00,0.83,u17581,United Kingdom,SET OF 12 FAIRY CAKE BAKING CASES
405060,581580,37500,1,2011-12-09 12:20:00,4.95,u12748,United Kingdom,TEA TIME TEAPOT IN GIFT BOX
405061,581585,22915,24,2011-12-09 12:31:00,0.19,u15804,United Kingdom,ASSORTED BOTTLE TOP MAGNETS


In [155]:
check.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399841 entries, 0 to 405062
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       399841 non-null  object        
 1   stock_code    399841 non-null  object        
 2   quantity      399841 non-null  int64         
 3   invoice_date  399841 non-null  datetime64[ns]
 4   price         399841 non-null  float64       
 5   customer_id   399841 non-null  object        
 6   country       399841 non-null  object        
 7   description   399841 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.5+ MB


In [157]:
# check - reading an s3 file

key = 'friday_transformations/sh_online_transactions_v2.csv'

s3_client = connect_to_s3(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

response = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=key)

# if you are reading a pickle file use read pickle, otherwise use read csv
#check = pd.read_pickle(response.get("Body"))
check = pd.read_csv(response.get("Body"))

check.info()

# the csv file loses the data type for the invoice_date

Connection to s3 made
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       399841 non-null  object 
 1   stock_code    399841 non-null  object 
 2   quantity      399841 non-null  int64  
 3   invoice_date  399841 non-null  object 
 4   price         399841 non-null  float64
 5   customer_id   399841 non-null  object 
 6   country       399841 non-null  object 
 7   description   399841 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 24.4+ MB


## Loading Data to local data folder 

In [24]:
# make sure all transformations are in place
# anything else?!

online_trans_cleaned.to_pickle("../data/online_transactions_v2.pkl")